## **SETD2 with SAM**

## **Import modules**

In [1]:
import os
import sys

data_path=os.path.abspath('/media/dataHog/hca/SETD2') #Where your group data is
base_path=os.path.abspath('/media/dataHog/hca/SimFound_v2/source/') #Where your source code is (SFv2)

sys.path.append(base_path)
sys.path.append(data_path)

import importlib

import warnings
warnings.filterwarnings('ignore')
import Protocols as P

import main
import tools
import Featurize as F


from simtk.unit import *

## **Define Project**

In [2]:
importlib.reload(main)

workdir=data_path
results=workdir+'/results'
inputs=workdir+'inputs/structures/'

workdir=tools.Functions.fileHandler([workdir], _new=False)
tools.Functions.fileHandler([results, inputs])


#protein=['WT', 'R167Q', 'I602G']
protein=['setd2_open']

ligand=['SAM']
parameters=['310K']
timestep=20*picoseconds

project=main.Project(title='SETD2-SAM', 
                     hierarchy=('protein', 'ligand', 'parameter'), 
                     workdir=workdir,
                     parameter=parameters, 
                     replicas=20, 
                     protein=protein, 
                     ligand=ligand, 
                     results=results,
                     topology='wt_SETD2.pdb',
                     timestep=timestep,
                    initial_replica=16)


project_systems=project.setSystems()

Converted parameter "temperature" (in K) into scalar: 310.0 K
System setd2_open-SAM-310K-16 with parameter 310.0 K defined
top setd2_open-SAM-310K-16
System setd2_open-SAM-310K-17 with parameter 310.0 K defined
top setd2_open-SAM-310K-17
System setd2_open-SAM-310K-18 with parameter 310.0 K defined
top setd2_open-SAM-310K-18
System setd2_open-SAM-310K-19 with parameter 310.0 K defined
top setd2_open-SAM-310K-19
System setd2_open-SAM-310K-20 with parameter 310.0 K defined
top setd2_open-SAM-310K-20


## **Simulation Protocol**

## Set defaults

In [3]:

SETD2_variants={('A',1499): 'CYX',
                ('A',1501): 'CYX',
                ('A',1516): 'CYX',
                ('A',1520): 'CYX',
                ('A',1529): 'CYX',
                ('A',1533):'CYX',
                ('A',1539):'CYX',
                ('A',1631):'CYX',
                ('A',1678):'CYX',
                ('A',1680):'CYX',
                ('A',1685):'CYX'}

NPT_equilibration={'ensemble': 'NPT',
                'step': 5*nanoseconds, 
                'report': timestep, 
                'restrained_sets': {'selections': ['protein and backbone', 'resname ZNB', 'resname SAM'],
                                    'forces': [100*kilojoules_per_mole/angstroms, 150*kilojoules_per_mole/angstroms, 150*kilojoules_per_mole/angstroms]}}

NPT_production={'ensemble': 'NPT',
                'step': 500*nanoseconds, 
                'report': timestep}

## Setup simulation protocol

In [5]:
importlib.reload(P)
importlib.reload(tools)
extra_pdb = ['ZNB_open.pdb', 'SAM.pdb']
#smiles = 'C[S+](CC[C@@H](C(=O)[O-])N)C[C@@H]1[C@H]([C@H]([C@@H](O1)N2C=NC3=C(N=CN=C32)N)O)O'
xml_files = ['protein.ff14SB.xml', 'tip4pew.xml', 'ZNB.xml', 'SAM.xml', 'bonded.xml', 'gaff.xml']


simulation=P.Protocols(project, overwrite_mode=False)
    
simulation.pdb2omm(input_pdb=system.input_topology,
                                     extra_input_pdb=extra_pdb,
                                     ff_files=xml_files,
                                     protonate=True,
                                     solvate=True,
                                     pH_protein=7.0,
                                     residue_variants=SETD2_variants)
                                    #insert_molecules=True)
                                    #insert_smiles=smiles)
simulation.setSimulations(dt=0.002*picoseconds, 
                                  temperature = system.scalar,
                                  pH=7.0,
                                  friction = 1/picosecond,
                                  equilibrations=[NPT_equilibration],
                                  productions=[NPT_production],
                                  pressure=1*atmospheres)


Setting openMM simulation of setd2_open-SAM-310K-16 protocols in None
	Warning! Found conserved openMM system(s). None/omm_1.pkl
	Status open: 1
	Loading conserved openMM system:  None/omm_1.pkl


FileNotFoundError: [Errno 2] No such file or directory: 'None/system.pdb'

## **WARNING!** Run simulation(s)

In [ ]:
importlib.reload(P)
importlib.reload(tools)

gpu_index='0'

simulations={}
for name, simulation in simulated_systems.items():  
    simulation.setSimulations(dt=0.002*picoseconds, 
                                  temperature = system.scalar,
                                  pH=7.0,
                                  friction = 1/picosecond,
                                  equilibrations=[NPT_equilibration],
                                  productions=[NPT_production],
                                  pressure=1*atmospheres)
    
    
    simulation.runSimulations(run_productions=True,  
                              compute_time=2,
                              reportFactor=0.05,
                             gpu_index=gpu_index)        
    
    simulations[name]=simulation


In [ ]:
simulations